In [1]:
import pandas as pd
import numpy as np

In [2]:
label_df = pd.read_json('../data/icd-9_definitions/icd_category_ranges.json', 
                 orient='index').reset_index().rename({'index':'label_names'},axis='columns')
label_df['targets'] = [f'''target_{i}''' for i in range(label_df.shape[0])]
label_df

,label_names,min,max,targets
0,Infectious,1,139,target_0
1,Neoplasia,140,239,target_1
2,Endo-Immune,240,279,target_2
3,Blood,280,289,target_3
4,Mental,290,319,target_4
5,Nervous,320,359,target_5
6,Sense organs,360,389,target_6
7,Circulatory,390,459,target_7
8,Respiratory,460,519,target_8
9,Digestive,520,579,target_9


In [3]:
descriptions_df = pd.read_csv('../data/icd-9_definitions/D_ICD_DIAGNOSES.csv')
descriptions_df['CAT_INT'] = descriptions_df['ICD9_CODE'].apply(lambda x: int(x[:3]) if x.isdigit() else None)
descriptions_df = descriptions_df[descriptions_df['CAT_INT'].notna()]

In [4]:
# https://stackoverflow.com/questions/44367672/best-way-to-join-merge-by-range-in-pandas
a = descriptions_df.CAT_INT.values
bh = label_df['max'].values
bl = label_df['min'].values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

icd9_desc = pd.concat([
    descriptions_df.iloc[i, :].reset_index(drop=True),
    label_df.iloc[j, :].reset_index(drop=True)
], axis=1)


In [5]:
import re

regex = re.compile('[^a-zA-Z\s]')

class_desc = icd9_desc.groupby('targets')['LONG_TITLE'].apply(lambda x: regex.sub(''
                                                                                  ,' '.join(x).lower())).reset_index()

In [6]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
from natsort import natsorted

class_desc['LONG_TITLE'] = class_desc['LONG_TITLE'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
class_desc['LONG_TITLE'] = class_desc['LONG_TITLE'].apply(lambda x: ', '.join(pd.Series(x.split()).value_counts()[:25].index.values))
class_desc['targets'] = pd.Categorical(class_desc['targets'], natsorted(class_desc['targets'].values))
class_desc = class_desc.sort_values(by='targets')


In [7]:
import csv
class_desc.to_csv('../data/icd-9_definitions/class_descriptions.txt', sep=':', index=False, header=False, 
                  quotechar=' ')

In [11]:
label_df.label_names.str.lower().to_csv('../data/mimic/label_names.txt', index=False, header=False)